In [2]:
%load_ext autoreload
%autoreload 2

# 1. Data Preparation

## 1.1 Raw Data

### 1.1.1 Fetching

In [3]:
from os import path
import json
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [10]:

filename = path.join(path.abspath(''), "dbconnection.json")
try:s
    with open(filename, "r") as jsonfile:
        db_schema = json.load(jsonfile)

    raw = db_schema["RAW"]
    processing = db_schema["PROCESS"]
    analysis = db_schema["ANALYSIS"]

    raw_engine = create_engine(f"{raw['DIALECT']}://{raw['USER']}:{raw['PASSWORD']}@{raw['HOST']}:{raw['PORT']}/{raw['DBNAME']}", encoding='utf-8')
    processing_engine = create_engine(f"{processing['DIALECT']}://{processing['USER']}:{processing['PASSWORD']}@{processing['HOST']}:{processing['PORT']}/{processing['DBNAME']}", encoding='utf-8')
    analysis_engine = create_engine(f"{analysis['DIALECT']}://{analysis['USER']}:{analysis['PASSWORD']}@{analysis['HOST']}:{analysis['PORT']}/{analysis['DBNAME']}", encoding='utf-8')

except:
    print("Failed to parsing DB data from .json file.")


In [11]:
con_raw = raw_engine.connect()
df_gswitch4_event_info = pd.read_sql(f"""SELECT * FROM GSWITCH4_EVENT_INFO""", con_raw)
df_gswitch4_event_MJ_R = pd.read_sql(f"""SELECT * FROM GSWITCH4_EVENT_MJ_R""", con_raw)
df_protocol_spec = pd.read_sql(f"""SELECT * FROM PROTOCOL_SPEC""", con_raw)
df_protocol_spec_set = pd.read_sql(f"""SELECT * FROM PROTOCOL_SPEC_SET""", con_raw)
con_raw.close()

### 1.1.2 Rename

In [17]:
df_gswitch4_event_info.columns = df_gswitch4_event_info.columns.str.upper()
df_gswitch4_event_info = df_gswitch4_event_info[df_gswitch4_event_info["TIME_STEMP"]>"19700101000000000"]
df_gswitch4_event_info["TIME_STEMP"] = pd.to_datetime(df_gswitch4_event_info["TIME_STEMP"], format="%Y%m%d%H%M%S%f").view(np.int64).values
df_gswitch4_event_MJ_R.columns = df_gswitch4_event_MJ_R.columns.str.upper()
df_protocol_spec.columns = df_protocol_spec.columns.str.upper()
df_protocol_spec_set.columns = df_protocol_spec_set.columns.str.upper()

### 1.1.3 Inspecting

In [18]:
df_gswitch4_event_info.head()

,EVENT_NUM,TIME_STEMP,AREA_ID,D_GROUP_1_ID,D_GROUP_2_ID,DATA_TYPE_ID,DIRECTION,SAVE_TIME,ERROR,SWITCH_CNT
0,2216,1462778276000000000,4,6,8,8,L,2016-05-18 22:34:36,0,0
1,2535,1461743907290000000,4,2,10,8,L,2016-05-18 22:43:28,0,0
2,2536,1463424322000000000,4,16,5,8,R,2016-05-18 22:43:29,0,0
3,2537,1462979673910000000,4,22,7,9,R,2016-05-18 22:43:32,0,1
4,2538,1463466888100000000,4,38,6,8,L,2016-05-18 22:43:33,0,0


In [19]:
df_gswitch4_event_MJ_R.head()

,EVENT_NUM,EVENT_SEQ,L_KR_VOLT,R_KR_VOLT,OUT_VOLT,OUT_CURR
0,3348598,2,0.03,24.24,0.0,0.0
1,3348598,3,0.05,-8.45,0.0,0.0
2,3348598,4,0.03,-0.69,0.0,0.0
3,3348598,5,0.03,0.0,0.0,0.0
4,3348598,6,0.05,0.03,0.0,0.0


In [20]:
df_protocol_spec.head()

,PROTOCOL_ID,SYS_ID,SYS_TYPE_ID,D_GROUP_1_ID,D_GROUP_2_ID,AREA_ID,DATA_TYPE_ID,SEQ_ID,SEQ_SUB_ID,ITEM_NM,...,TOKEN_SIZE,TOKEN_SIZE_TYPE,TOKEN_TYPE,ITEM_DESC,RETURN_TYPE,MAPPING_USE,ALARM_TYPE,VALUE_USE,USE_GUBUN,TRACK_TYPE
0,3504,101,2,6,5,2,3,1,8,25,...,1,b,H,정위,INT,2,42,N,W,None
1,3505,101,2,6,6,2,3,1,1,26,...,1,b,H,사용안함,INT,2,0,N,W,None
2,3506,101,2,6,6,2,3,1,2,26,...,1,b,H,불일치,INT,3,46,N,W,None
3,3507,101,2,6,6,2,3,1,3,26,...,1,b,H,고장,INT,3,53,N,W,None
4,3508,101,2,6,6,2,3,1,4,26,...,1,b,H,철사쇄정,INT,2,45,N,W,None


In [21]:
mjns = lambda x: "NS" if x in [1,2,3,4,5,11] else ("MJ" if x in [6,7,8,9,10,12] else "UNKNOWN")
fr = lambda x: "R" if x in [1, 3, 6, 8] else ("F" if x in [2,4,5,7,9,10])


In [29]:
df_protocol_spec_copy = df_protocol_spec[df_protocol_spec["SYS_ID"].isin([206])].copy()
df_protocol_spec_copy = df_protocol_spec_copy[["SYS_ID", "SYS_TYPE_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "AREA_ID", "DATA_TYPE_ID"]].drop_duplicates().reset_index(drop=True)
df_protocol_spec_copy["MJNS"] = df_protocol_spec_copy["DATA_TYPE_ID"].apply(mjns)
df_protocol_spec_copy = df_protocol_spec_copy[df_protocol_spec_copy.groupby(["SYS_ID", "SYS_TYPE_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "AREA_ID"])["MJNS"].transform("nunique")>1]
pd.merge(df_protocol_spec_copy, device_name, left_on = ["SYS_ID", "SYS_TYPE_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "AREA_ID"], right_on = ["SYS_ID", "SYS_TYPE_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "AREA_ID"], how="inner")

,SYS_ID,SYS_TYPE_ID,D_GROUP_1_ID,D_GROUP_2_ID,AREA_ID,DATA_TYPE_ID,MJNS,DEVICE_NAME
0,206,1,15,3,4,1,NS,대전_31B
1,206,1,15,3,4,3,NS,대전_31B
2,206,1,15,3,4,11,MJ,대전_31B
3,206,1,15,3,4,2,NS,대전_31B
4,206,1,15,3,4,4,NS,대전_31B
...,...,...,...,...,...,...,...,...
200,206,1,5,1,4,1,NS,대전_N23B
201,206,1,5,1,4,3,NS,대전_N23B
202,206,1,5,1,4,11,MJ,대전_N23B
203,206,1,5,1,4,2,NS,대전_N23B


In [41]:
df_protocol_spec_set.head()

,PROTOCOL_ID,SYS_NM,SYS_TYPE_NM,D_GROUP_1_NM,D_GROUP_2_NM,AREA_NM,DATA_TYPE_NM,LEV,UI_USE,ALARM_USE,...,UPPER_LIMIT_D,LOWER_LIMIT_D,USE_MAPP_D,UPPER_LIMIT_DL,LOWER_LIMIT_DL,USE_MAPP_DL,CREATE_DATE,AL_DATE,CHART_MIN,CHART_MAX
0,209760,전자연동장치,혁신,진로구분정보,21L-D,신탄진,None,1,Y,N,...,0,0,0,0,0,0.0,20161206150854501,20161206150854501,0,0
1,209761,전자연동장치,혁신,진로구분정보,21L-D,신탄진,None,1,Y,N,...,0,0,0,0,0,0.0,20161206150854501,20161206150854501,0,0
2,209762,전자연동장치,혁신,진로구분정보,21L-D,신탄진,None,1,Y,N,...,0,0,0,0,0,0.0,20161206150854501,20161206150854501,0,0
3,209763,전자연동장치,혁신,진로구분정보,21L-7DN,신탄진,None,1,Y,N,...,0,0,0,0,0,0.0,20161206150854501,20161206150854501,0,0
4,209764,전자연동장치,혁신,진로구분정보,21L-7DN,신탄진,None,1,Y,N,...,0,0,0,0,0,0.0,20161206150854501,20161206150854501,0,0


## 1.2 Preprocessing

### 1.2.1 Device Sharding

In [25]:
dev_valid = [206] # GSWITCH

In [26]:
df_valid_protocol_spec = df_protocol_spec[df_protocol_spec['SYS_ID'].isin(dev_valid)].reset_index(drop=True)
df_valid_protocol_spec_set = df_protocol_spec_set[df_protocol_spec_set['PROTOCOL_ID'].isin(df_valid_protocol_spec["PROTOCOL_ID"])].reset_index(drop=True)
merged = pd.merge(df_valid_protocol_spec, df_valid_protocol_spec_set, left_on=["PROTOCOL_ID"], right_on=["PROTOCOL_ID"], how="inner", suffixes=('', '_DUP'))

device_type = merged[["SYS_ID", "SYS_NM", "SYS_TYPE_ID", "SYS_TYPE_NM"]].drop_duplicates()
device_type = device_type.fillna("")
device_type["DEVICE_TYPE"] = device_type["SYS_NM"] + "_" +  device_type["SYS_TYPE_NM"]
device_type = device_type[["DEVICE_TYPE", "SYS_ID", "SYS_TYPE_ID"]].reset_index(drop=True)

device_name = merged[["SYS_ID", "SYS_TYPE_ID", "AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "AREA_NM", "D_GROUP_1_NM", "D_GROUP_2_NM"]].drop_duplicates()
device_name = device_name.fillna("")
device_name["DEVICE_NAME"] = device_name["AREA_NM"] + "_" + device_name["D_GROUP_1_NM"] + device_name["D_GROUP_2_NM"]
device_name = device_name[["DEVICE_NAME", "SYS_ID", "SYS_TYPE_ID", "AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID"]].reset_index(drop=True)

device_data_type = merged[["SYS_ID", "SYS_TYPE_ID", "AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "DATA_TYPE_ID", "SEQ_ID", "SEQ_SUB_ID", "DATA_TYPE_NM", "ITEM_DESC"]].drop_duplicates()
device_data_type = device_data_type.fillna("")
device_data_type["DEVICE_DATA_TYPE"] = device_data_type["DATA_TYPE_NM"] +  "_" + device_data_type["ITEM_DESC"]
device_data_type = device_data_type[["DEVICE_DATA_TYPE", "SYS_ID", "SYS_TYPE_ID", "AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "DATA_TYPE_ID", "SEQ_ID", "SEQ_SUB_ID"]].reset_index(drop=True)

df_result_protocol_id = pd.merge(df_valid_protocol_spec, device_data_type, left_on=["SYS_ID", "SYS_TYPE_ID", "AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "DATA_TYPE_ID", "SEQ_ID", "SEQ_SUB_ID"], right_on = ["SYS_ID", "SYS_TYPE_ID", "AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID", "DATA_TYPE_ID", "SEQ_ID", "SEQ_SUB_ID"], how="inner", suffixes=('', '_DUP'))
df_result_protocol_id = pd.merge(df_result_protocol_id, device_name, left_on=["SYS_ID", "SYS_TYPE_ID", "AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID"], right_on = ["SYS_ID", "SYS_TYPE_ID", "AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID"], how="inner", suffixes=('', '_DUP'))
df_result_protocol_id = pd.merge(df_result_protocol_id, device_type, left_on=["SYS_ID", "SYS_TYPE_ID"], right_on = ["SYS_ID", "SYS_TYPE_ID"], how="inner", suffixes=('', '_DUP'))
df_result_protocol_id = df_result_protocol_id[["PROTOCOL_ID", "DEVICE_TYPE", "DEVICE_NAME", "DEVICE_DATA_TYPE"]]

In [28]:
device_name

,DEVICE_NAME,SYS_ID,SYS_TYPE_ID,AREA_ID,D_GROUP_1_ID,D_GROUP_2_ID
0,대전_N35P,206,1,4,19,5
1,대전_31B,206,1,4,15,3
2,대전_31A,206,1,4,15,2
3,대전_N32P,206,1,4,16,5
4,대전_N35F,206,1,4,19,6
...,...,...,...,...,...,...
124,대전_26A,206,1,4,9,2
125,대전_30A,206,1,4,14,2
126,대전_25A,206,1,4,8,2
127,대전_26B,206,1,4,9,3


### 1.2.2 Matching event to device

- 1	NS 기계실 실시간 검측값
- 2	NS 현장 실시간 검측값
- 3	NS 기계실 전환 이벤트 검측값
- 4	NS 현장 전환 이벤트 검측값 (회로제어기 출력)
- 5	NS 현장 전환 이벤트 검측값 (회로제어기 정위 출력, 반위 출력)
- 6	MJ81 기계실 실시간 검측값
- 7	MJ81 현장 실시간 검측값
- 8	MJ81 기계실 이벤트 검측값 
- 9	MJ81 현장 단상 이벤트 검측값
- 10	MJ81 현장 삼상 이벤트 검측값
- 11	NS 실시간 접점 정보
- 12	MJ81 실시간 접점 정보
- 13	DIM Master/Slave정보
- 14	선로전환기 방향 불일치정보

In [ ]:
mjns = lambda x: "NS" if x <= 5 else "MJ"
fr = lambda x: "F" if x in [4, 5, 7, 9] else "R"


In [46]:
df_gswitch_event_device = pd.merge(df_gswitch4_event_info, device_name, left_on=["AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID"], right_on=["AREA_ID", "D_GROUP_1_ID", "D_GROUP_2_ID"], how="inner")
df_gswitch_event_device["MJNS"] = df_gswitch_event_device["SYS_TYPE_ID"].apply(lambda x: "NS" if x <= 5 else "MJ")
df_gswitch_event_device["FR"] = df_gswitch_event_device["SYS_TYPE_ID"].apply(lambda x: "F" if x in [4, 5, 7, 9] else "R")
df_gswitch_event_device = df_gswitch_event_device[["DEVICE_NAME", "TIME_STEMP", "EVENT_NUM", "ERROR", "DIRECTION", "MJNS", "FR"]]
df_gswitch_event_device

,DEVICE_NAME,TIME_STEMP,EVENT_NUM,ERROR,DIRECTION,MJNS,FR
0,대전_N24AF,1462778276000000000,2216,0,L,NS,R
1,대전_N24AF,1462780425360000000,2623,0,L,NS,R
2,대전_N24AF,1462942497120000000,4133,0,L,NS,R
3,대전_N24AF,1461818914060000000,4154,0,L,NS,R
4,대전_N24AF,1461822757820000000,4161,0,R,NS,R
...,...,...,...,...,...,...,...
3568985,대전_40A,1595616010430000000,3600341,0,R,NS,R
3568986,대전_40A,1595792431330000000,3605170,0,N,NS,R
3568987,대전_40A,1595792430930000000,3605171,0,N,NS,R
3568988,대전_40A,1595599781230000000,3599763,0,R,NS,R


In [47]:
df_gswitch_event_device[df_gswitch_event_device["ERROR"]==0]

,DEVICE_NAME,TIME_STEMP,EVENT_NUM,ERROR,DIRECTION,MJNS,FR
0,대전_N24AF,1462778276000000000,2216,0,L,NS,R
1,대전_N24AF,1462780425360000000,2623,0,L,NS,R
2,대전_N24AF,1462942497120000000,4133,0,L,NS,R
3,대전_N24AF,1461818914060000000,4154,0,L,NS,R
4,대전_N24AF,1461822757820000000,4161,0,R,NS,R
...,...,...,...,...,...,...,...
3568985,대전_40A,1595616010430000000,3600341,0,R,NS,R
3568986,대전_40A,1595792431330000000,3605170,0,N,NS,R
3568987,대전_40A,1595792430930000000,3605171,0,N,NS,R
3568988,대전_40A,1595599781230000000,3599763,0,R,NS,R


In [48]:
df_gswitch_event_device[df_gswitch_event_device["ERROR"]!=0]

,DEVICE_NAME,TIME_STEMP,EVENT_NUM,ERROR,DIRECTION,MJNS,FR


In [ ]:
from dataclasses import dataclass
from datetime import date

@dataclass
class Event:
    timestamp: int # Unix time
    x: list[int] # array of 
    y: int = 0

@dataclass
class Dev:
    device_name: str # 대전_N24AF
    MJNS : str # MJ, NS
    FR : str # Field, Room
        
@dataclass
class DevEvents:
    device : Dev
    event_type: str # L_KR_VOLT
    events: list[Event]
    

There is no error data on its DB

### 1.2.3 Error data Cluster

### 1.2.4 Label device event data to alarm cluster

### 1.2.5 Translate event data as ndarray

In [ ]:
events = pd.unique(df_gswitch4_event_MJ_R["EVENT_NUM"])
evlen = len(events)
X_L_V=np.zeros([evlen, 1000])
X_R_V=np.zeros([evlen, 1000])
X_O_V=np.zeros([evlen, 1000])
X_O_I=np.zeros([evlen, 1000])
Y = np.zeros([evlen, 1])

for idx, event in enumerate(events):
    df_event=df_gswitch4_event_MJ_R[df_gswitch4_event_MJ_R["EVENT_NUM"]==event].sort_values(by="EVENT_SEQ")
    X_L_V[idx]=df_event["L_KR_VOLT"]
    X_R_V[idx]=df_event["R_KR_VOLT"]
    X_O_V[idx]=df_event["OUT_VOLT"]
    X_O_I[idx]=df_event["OUT_CURR"]
    Y[idx]=df_event["CLUSTER_ID"]

In [ ]:
pd.merge(df_gswitch_event_device, df_gswitch4_event_MJ_R, left_on="EVENT_NUM", right_on="EVENT_NUM", how="inner")[[]]

## 1.3 Data Save

# 2. Review